In [ ]:
#| default_exp meiosis

In [ ]:
#| export


import jax
import jax.numpy as jnp
from jax import lax
from functools import partial
from typing import Tuple
from chewc.structs import GeneticMap

@partial(jax.jit, static_argnames=("max_crossovers",))
def create_gamete(
    key: jax.Array,
    parent_geno: jax.Array,  # (n_chr, 2, n_loci)
    parent_ibd: jax.Array,   # (n_chr, 2, n_loci)
    genetic_map: GeneticMap,
    interference_nu: float,
    max_crossovers: int,
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """Return a haploid gamete (geno, ibd) of shape (n_chr, n_loci)."""

    def _sample_chiasmata(k: jax.Array, map_length: jnp.ndarray, nu: float, L: int):
        """Renewal process with gamma-distributed inter-chiasma distances."""
        shape, scale = nu, 1.0 / (2.0 * nu)

        def body(carry, _):
            kk, last_pos = carry
            kk, sub = jax.random.split(kk)
            step = jax.random.gamma(sub, shape) * scale
            pos = last_pos + step
            return (kk, pos), pos

        k, start_k = jax.random.split(k)
        start = jax.random.uniform(start_k, (), minval=-10.0, maxval=0.0)

        (_, _), positions = lax.scan(body, (k, start), None, length=L)
        valid = (positions > 0.0) & (positions < map_length)
        sentinel = map_length + 1.0
        # Keep shape static, sort to ensure monotonic positions
        positions = jnp.where(valid, positions, sentinel)
        positions = jnp.sort(positions)
        return positions  # (L,)

    def _recombine_chr(
        k: jax.Array,
        chr_geno: jnp.ndarray,   # (2, n_loci)
        chr_ibd: jnp.ndarray,    # (2, n_loci)
        loci_pos: jnp.ndarray,   # (n_loci,)
        nu: float,
        L: int,
    ):
        k_chiasma, k_hap = jax.random.split(k)
        cross_pos = _sample_chiasmata(k_chiasma, loci_pos[-1], nu, L)         # (L,)
        cross_idx = jnp.searchsorted(loci_pos, cross_pos, side="left")        # (L,)
        # segments = number of crossovers <= each locus index
        segments = jnp.searchsorted(cross_idx, jnp.arange(loci_pos.shape[0]), side="right")
        start_hap = jax.random.randint(k_hap, (), 0, 2, dtype=jnp.int32)
        hap_choice = (start_hap + segments) & 1  # 0/1 per locus

        gam_geno = jnp.where(hap_choice == 0, chr_geno[0], chr_geno[1])  # (n_loci,)
        gam_ibd  = jnp.where(hap_choice == 0, chr_ibd[0],  chr_ibd[1])   # (n_loci,)
        return gam_geno, gam_ibd

    n_chr = parent_geno.shape[0]
    chr_keys = jax.random.split(key, num=n_chr)
    vmapped = jax.vmap(_recombine_chr, in_axes=(0, 0, 0, 0, None, None))
    gamete_geno, gamete_ibd = vmapped(
        chr_keys, parent_geno, parent_ibd, genetic_map.locus_positions, interference_nu, max_crossovers
    )
    return gamete_geno, gamete_ibd  # (n_chr, n_loci), (n_chr, n_loci)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()